<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P4_Qlora_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#llama： 前面也是都跟之前的llama训练一样，导入模型的时候就不一样了
model = AutoModelForCausalLM.from_pretrained("D:/Pretrained_models/modelscope/Llama-2-13b-ms", low_cpu_mem_usage=True,
                                             torch_dtype=torch.half, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.half,
                                             bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)
#bnb_4bit_compute_dtype=torch.half这个半精度可能会有溢出的问题，如果可以的话还是建议写bf16
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

In [ ]:
model.enable_input_require_grads()
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    logging_steps=10,
    num_train_epochs=1,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit" #这个就是qlora的分页优化器，还有paged_adamw_8bit，paged_lion_32bit/8bit
)
#可以点击TrainingArguments查看都有哪些参数，有一个optimizernames，里面带BNB或者8BIT的都是bitsandbytes提供的优化器，其他都是Huggingface自带的

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
#chatglm3：同理
model = AutoModelForCausalLM.from_pretrained("d:/Pretrained_models/ZhipuAI/chatglm3-6b-base/", trust_remote_code=True, low_cpu_mem_usage=True,
                                             torch_dtype=torch.half, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.half,
                                             bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)
#bnb_4bit_compute_dtype=torch.half这个半精度可能会有溢出的问题，如果可以的话还是建议写bf16
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

model.enable_input_require_grads()
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    logging_steps=10,
    num_train_epochs=1,
    learning_rate=1e-4,
    remove_unused_columns=False,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit"#这个就是qlora的分页优化器，还有paged_adamw_8bit，paged_lion_32bit/8bit
)
#可以点击TrainingArguments查看都有哪些参数，有一个optimizernames，里面带BNB或者8BIT的都是bitsandbytes提供的优化器，其他都是Huggingface自带的
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

internlm20B只需要14G的显存；llama2的13B模型只需要约8-9个G显存；chatglm3的6B模型只需要5-6个G的显存